In [80]:
import logging
import pkgutil
import tarfile
import json
from io import BytesIO, StringIO
from typing import Optional, Tuple, Union, Dict

import numpy as np
import pandas as pd
import requests
from requests import RequestException
from sklearn.preprocessing import LabelEncoder

from alibi.utils.data import Bunch

logger = logging.getLogger(__name__)

ADULT_URLS = ['https://storage.googleapis.com/seldon-datasets/adult/adult.data',
              'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
              'http://mlr.cs.umass.edu/ml/machine-learning-databases/adult/adult.data']

GERMAN_CREDIT_URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data'

def fetch_german_credit(return_X_y: bool = False) -> Union[Bunch, Tuple[np.ndarray, np.ndarray]]:
    """
    Loads and pre-processes the 'german_credit' dataset.
    More info: http://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29

    Parameters
    ----------
    return_X_y : bool, default=False
        If True, return features `X` and labels `y` as numpy arrays.
        If False, return a Bunch object.

    Returns
    -------
    Bunch
        Dataset, labels, a list of features, and a dictionary containing a list with the potential categories
        for each categorical feature where the key refers to the feature column.
    (data, target) : tuple
        Only if return_X_y=True.
    """

    # Load the data from the provided file
    data = pd.read_csv('datasets/german.data', delimiter=' ', header=None)

    # Extract features and labels
    features = data.iloc[:, :-1]
    labels = data.iloc[:, -1].values - 1  # Adjusting the labels to be 0 (Good) and 1 (Bad)
    
    # Categorical columns indices based on dataset description
    categorical_cols = [0, 2, 3, 5, 6, 8, 9, 11, 13, 14, 16, 18, 19]
    
    # Map for storing category data
    category_map = {}
    for col in categorical_cols:
        le = LabelEncoder()
        features[col] = le.fit_transform(features[col].values)
        category_map[col] = list(le.classes_)

    # Convert features to numpy array for consistency with the other fetch function
    features = features.values
    
    # Feature and target names
    feature_names = [
        'ExistingChecking', 'Duration', 'CreditHistory', 'Purpose', 'CreditAmount', 'SavingsAccount',
        'EmploymentSince', 'InstallmentRatePercentage', 'PersonalStatusSex', 'OtherDebtors', 'PresentResidenceSince',
        'Property', 'Age', 'OtherInstallmentPlans', 'Housing', 'ExistingCreditsAtBank', 'Job', 
        'PeopleLiableToProvideMaintenance', 'Telephone', 'ForeignWorker'
    ]
    target_names = ['Good', 'Bad']  # Based on dataset description: 1 = Good, 2 = Bad

    if return_X_y:
        return features, labels

    return Bunch(data=features, target=labels, feature_names=feature_names, target_names=target_names, category_map=category_map)


In [63]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import Bunch
from typing import Union, Tuple
import numpy as np

def fetch_german_credit(return_X_y: bool = False) -> Union[Bunch, Tuple[np.ndarray, np.ndarray]]:
    """
    Loads and pre-processes the 'german_credit' dataset.
    More info: http://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29

    Parameters
    ----------
    data_path
        Path to the 'german.data' file.
    return_X_y
        If ``True``, return features `X` and labels `y` as `numpy` arrays. If ``False`` return a `Bunch` object.

    Returns
    -------
    Bunch
        Dataset, labels, a list of features and a dictionary containing a list with the potential categories
        for each categorical feature where the key refers to the feature column.
    (data, target)
        Tuple if ``return_X_y=True``
    """
    
    # Load the data from the provided file
    data = pd.read_csv('datasets/german.data', delimiter=' ', header=None)
    
    # Extract features and labels
    features = data.iloc[:, :-1]
    labels = data.iloc[:, -1].values - 1  # Adjusting the labels to be 0 (Good) and 1 (Bad)
    
    # Categorical columns indices based on dataset description
    categorical_cols = [0, 2, 3, 5, 6, 8, 9, 11, 13, 14, 16, 18, 19]
    
    # Feature names
    feature_names = ["Status of existing checking account", "Duration in month", "Credit history", "Purpose",
                     "Credit amount", "Savings account/bonds", "Present employment since", 
                     "Installment rate in percentage of disposable income", "Personal status and sex", 
                     "Other debtors / guarantors", "Present residence since", "Property", "Age in years",
                     "Other installment plans", "Housing", "Number of existing credits at this bank",
                     "Job", "Number of people being liable to provide maintenance for", "Telephone", "foreign worker"]
    
    # Map for storing category data
    category_map = {}
    
    # Mapping dictionary based on the provided data description
    mapping_dict = {
        0: {"A11": "... < 0 DM", "A12": "0 <= ... < 200 DM", "A13": "... >= 200 DM / salary assignments for at least 1 year", "A14": "no checking account"},
        2: {"A30": "no credits taken/ all credits paid back duly", "A31": "all credits at this bank paid back duly", "A32": "existing credits paid back duly till now", "A33": "delay in paying off in the past", "A34": "critical account/ other credits existing (not at this bank)"},
        3: {"A40": "car (new)", "A41": "car (used)", "A42": "furniture/equipment", "A43": "radio/television", "A44": "domestic appliances", "A45": "repairs", "A46": "education", "A47": "(vacation - does not exist?)", "A48": "retraining", "A49": "business", "A410": "others"},
        5: {"A61": "... < 100 DM", "A62": "100 <= ... < 500 DM", "A63": "500 <= ... < 1000 DM", "A64": ".. >= 1000 DM", "A65": "unknown/ no savings account"},
        6: {"A71": "unemployed", "A72": "... < 1 year", "A73": "1 <= ... < 4 years", "A74": "4 <= ... < 7 years", "A75": ".. >= 7 years"},
        9: {"A91": "male : divorced/separated", "A92": "female : divorced/separated/married", "A93": "male : single", "A94": "male : married/widowed", "A95": "female : single"},
        10: {"A101": "none", "A102": "co-applicant", "A103": "guarantor"},
        12: {"A121": "real estate", "A122": "if not A121 : building society savings agreement/ life insurance", "A123": "if not A121/A122 : car or other, not in attribute 6", "A124": "unknown / no property"},
        14: {"A141": "bank", "A142": "stores", "A143": "none"},
        15: {"A151": "rent", "A152": "own", "A153": "for free"},
        17: {"A171": "unemployed/ unskilled - non-resident", "A172": "unskilled - resident", "A173": "skilled employee / official", "A174": "management/ self-employed/ highly qualified employee/ officer"},
        19: {"A191": "none", "A192": "yes, registered under the customers name"},
#         20: {"A201": "yes", "A202": "no"}
    }
    
    # Apply mapping based on the mapping_dict
    for col, mapping in mapping_dict.items():
        features[col] = features[col].map(mapping)
    
    for col in categorical_cols:
        le = LabelEncoder()
        features[col] = le.fit_transform(features[col].values)
        category_map[feature_names[col]] = list(le.classes_)
    
    # Convert features to numpy array for consistency with the other fetch function
    features = features.values
    
    if return_X_y:
        return features, labels

    return Bunch(data=features, target=labels, feature_names=feature_names, category_map=category_map)


In [65]:
german = fetch_german_credit()

In [ ]:
def fetch_german_credit(return_X_y: bool = False) -> Union[Bunch, Tuple[np.ndarray, np.ndarray]]:
    """
    Loads and pre-processes the 'german_credit' dataset.

    Parameters
    ----------
    return_X_y: bool
        If ``True``, return features `X` and labels `y` as `numpy` arrays. If ``False`` return a `Bunch` object.
    file_path: str
        Path to the german.data file.

    Returns
    -------
    Bunch
        Dataset, labels, a list of features and a dictionary containing a list with the potential categories
        for each categorical feature where the key refers to the feature column.
    (data, target)
        Tuple if ``return_X_y=True``
    """
    # Mapping categorical columns based on provided description
    mapping_dict = {
    "ExistingChecking": {
        'A11': "... < 0 DM", 'A12': "0 <= ... < 200 DM", 'A13': "... >= 200 DM / salary assignments for at least 1 year",
        'A14': "no checking account"
    },
    "CreditHistory": {
        'A30': "no credits taken/ all credits paid back duly", 'A31': "all credits at this bank paid back duly",
        'A32': "existing credits paid back duly till now", 'A33': "delay in paying off in the past",
        'A34': "critical account/ other credits existing (not at this bank)"
    },
    "Purpose": {
        'A40': "car (new)", 'A41': "car (used)", 'A42': "furniture/equipment", 'A43': "radio/television",
        'A44': "domestic appliances", 'A45': "repairs", 'A46': "education", 'A47': "(vacation - does not exist?)",
        'A48': "retraining", 'A49': "business", 'A410': "others"
    },
    "SavingsAccount": {
        'A61': "... < 100 DM", 'A62': "100 <= ... < 500 DM", 'A63': "500 <= ... < 1000 DM", 'A64': "... >= 1000 DM",
        'A65': "unknown/ no savings account"
    },
    "EmploymentSince": {
        'A71': "unemployed", 'A72': "... < 1 year", 'A73': "1 <= ... < 4 years", 'A74': "4 <= ... < 7 years", 'A75': "... >= 7 years"
    },
    "PersonalStatusSex": {
        'A91': "male : divorced/separated", 'A92': "female : divorced/separated/married", 'A93': "male : single",
        'A94': "male : married/widowed", 'A95': "female : single"
    },
    "OtherDebtors": {
        'A101': "none", 'A102': "co-applicant", 'A103': "guarantor"
    },
    "Property": {
        'A121': "real estate", 'A122': "if not A121 : building society savings agreement/ life insurance",
        'A123': "if not A121/A122 : car or other, not in attribute 6", 'A124': "unknown / no property"
    },
    "Other installment plans": {
        'A141': "bank", 'A142': "stores", 'A143': "none"
    },
    "Housing": {
        'A151': "rent", 'A152': "own", 'A153': "for free"
    },
    "Job": {
        'A171': "unemployed/ unskilled - non-resident", 'A172': "unskilled - resident",
        'A173': "skilled employee / official", 'A174': "management/ self-employed/ highly qualified employee/ officer"
    },
    "Telephone": {
        'A191': "none", 'A192': "yes, registered under the customers name"
    },
    "foreign worker": {
        'A201': "yes", 'A202': "no"
    }
    }
#      data = pd.read_csv('datasets/german.data', delimiter=' ', header=None)
    # Load the data from the provided file
    raw_data = pd.read_csv('datasets/german.data', delimiter=' ', header=None)
    
    # Extract features and labels
    features = raw_data.iloc[:, :-1]
    labels = raw_data.iloc[:, -1].values - 1  # Adjusting the labels to be 0 (Good) and 1 (Bad)
    
    # Map for storing category data
    category_map = {}
    
    # Apply the mapping created earlier
    for col, mapping in german_feature_mappings.items():
        features[col] = features[col].map(mapping)
    
    # Convert categorical features to encoded labels
    categorical_cols = [0, 2, 3, 5, 6, 8, 9, 11, 13, 14, 16, 18, 19]
    
    for col in categorical_cols:
        le = LabelEncoder()
        features[col] = le.fit_transform(features[col].values)
        category_map[col] = list(le.classes_)
    
    # Convert features to numpy array for consistency
    features = features.values
    
    # Feature and target names
    feature_names = [
        'ExistingChecking', 'Duration', 'CreditHistory', 'Purpose', 'CreditAmount', 'SavingsAccount',
        'EmploymentSince', 'InstallmentRatePercentage', 'PersonalStatusSex', 'OtherDebtors', 'PresentResidenceSince',
        'Property', 'Age', 'OtherInstallmentPlans', 'Housing', 'ExistingCreditsAtBank', 'Job', 
        'PeopleLiableToProvideMaintenance', 'Telephone', 'ForeignWorker'
    ]
    target_names = ['Good', 'Bad']
    
    if return_X_y:
        return features, labels

    return Bunch(data=features, target=labels, feature_names=feature_names, target_names=target_names, category_map=category_map)

# Test the updated function
data_bunch = fetch_german_credit()
data_bunch.data[:5], data_bunch.feature_names, data_bunch.category_map


In [66]:
import os
import numpy as np
import pandas as pd
from copy import deepcopy
from typing import List, Tuple, Dict, Callable

import tensorflow as tf
import tensorflow.keras as keras

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from alibi.explainers import CounterfactualRLTabular, CounterfactualRL
# from alibi.datasets import fetch_adult
from alibi.models.tensorflow import HeAE
from alibi.models.tensorflow import Actor, Critic
# from alibi.models.tensorflow import ADULTEncoder, ADULTDecoder
from alibi.explainers.cfrl_base import Callback
from alibi.explainers.backends.cfrl_tabular import get_he_preprocessor, get_statistics, \
    get_conditional_vector, apply_category_mapping

In [67]:
import tensorflow as tf
from tensorflow import keras
from typing import List

class GermanCreditEncoder(keras.Model):
    """
    German Credit encoder used for Counterfactual with Reinforcement Learning experiments. The model consists of
    two fully connected layers with ReLU and tanh nonlinearities. The tanh nonlinearity clips the embedding in [-1, 1]
    as required in certain algorithms. The layers' dimensions can be set as desired, but for this example, 
    we'll use dimensions similar to the ADULTEncoder.
    """

    def __init__(self, hidden_dim: int, latent_dim: int, **kwargs):
        """
        Constructor.

        Parameters
        ----------
        hidden_dim
            Hidden dimension.
        latent_dim
            Latent dimension.
        """
        super().__init__(**kwargs)
        self.fc1 = keras.layers.Dense(hidden_dim)
        self.fc2 = keras.layers.Dense(latent_dim)

    def call(self, x: tf.Tensor, **kwargs) -> tf.Tensor:
        """
        Forward pass.

        Parameters
        ----------
        x
            Input tensor.
        **kwargs
            Other arguments.

        Returns
        -------
        Encoding representation having each component in the interval [-1, 1].
        """
        x = tf.nn.relu(self.fc1(x))
        x = tf.nn.tanh(self.fc2(x))
        return x

# For the decoder, we'll first need to determine the output dimensions based on the number of unique values 
# for each categorical variable in the german_credit dataset. Let's calculate those dimensions.


In [81]:
# Calculate the output dimensions for the GermanCreditDecoder based on the number of unique values 
# for each categorical variable in the german_credit dataset

# Recall the categorical columns indices for the german_credit dataset
categorical_cols = [0, 2, 3, 5, 6, 8, 9, 11, 13, 14, 16, 18, 19]

# Calculate the unique values for each categorical column
output_dims = [len(german_credit_bunch.category_map[col]) for col in categorical_cols]

# The first dimension will be for the numerical features. 
# We obtain it by subtracting the number of categorical columns from the total number of columns.
output_dims.insert(0, german_data.shape[1] - len(categorical_cols) - 1)  # -1 to exclude the target column

output_dims


[6, 4, 5, 10, 5, 5, 4, 3, 4, 3, 3, 4, 2, 2]

In [69]:
# Defining the GermanCreditDecoder class

class GermanCreditDecoder(keras.Model):
    """
    German Credit decoder used for Counterfactual with Reinforcement Learning experiments. The model consists of
    a fully connected layer with ReLU nonlinearity, and a multiheaded layer, one for each categorical feature and
    a single head for the rest of numerical features. 
    """

    def __init__(self, hidden_dim: int, output_dims: List[int], **kwargs):
        """
        Constructor.

        Parameters
        ----------
        hidden_dim
            Hidden dimension.
        output_dims
            List of output dimensions, one for each categorical variable and one for all numerical variables combined.
        """
        super().__init__(**kwargs)

        self.fc1 = keras.layers.Dense(hidden_dim)
        self.fcs = [keras.layers.Dense(dim) for dim in output_dims]

    def call(self, x: tf.Tensor, **kwargs) -> List[tf.Tensor]:
        """
        Forward pass.

        Parameters
        ----------
        x
            Input tensor.
        **kwargs
            Other arguments. Not used.

        Returns
        -------
        List of reconstructions of the input tensor. The first element corresponds to the reconstruction 
        of all the numerical features, and the subsequent elements correspond to each categorical feature.
        """
        x = tf.nn.relu(self.fc1(x))
        xs = [fc(x) for fc in self.fcs]
        return xs

# The defined GermanCreditEncoder and GermanCreditDecoder can be used similarly to the provided ADULTEncoder and ADULTDecoder.


In [82]:
# Fetch adult dataset
german = fetch_german_credit()

# Separate columns in numerical and categorical.
categorical_names = [german.feature_names[i] for i in german.category_map.keys()]
categorical_ids = list(german.category_map.keys())

numerical_names = [name for i, name in enumerate(german.feature_names) if i not in german.category_map.keys()]
numerical_ids = [i for i in range(len(german.feature_names)) if i not in german.category_map.keys()]


# german = fetch_german_credit()

# # Separate columns in numerical and categorical.
# categorical_ids = [german.feature_names.index(key) for key in german.category_map.keys()]
# categorical_names = [german.feature_names[i] for i in categorical_ids]

# numerical_ids = [i for i in range(len(german.feature_names)) if i not in categorical_ids]
# numerical_names = [name for i, name in enumerate(german.feature_names) if i not in categorical_ids]

# Split data into train and test
X, Y = german.data, german.target
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=13)


# Split data into train and test
X, Y = german.data, german.target
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=13)

In [83]:
# Define numerical standard scaler.
num_transf = StandardScaler()

# Define categorical one-hot encoder.
cat_transf = OneHotEncoder(
    categories=[range(len(x)) for x in german.category_map.values()],
    handle_unknown="ignore"
)

# Define column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", cat_transf, categorical_ids),
        ("num", num_transf, numerical_ids),
    ],
    sparse_threshold=0
)
# Fit preprocessor.
preprocessor.fit(X_train)

# Preprocess train and test dataset.
X_train_ohe = preprocessor.transform(X_train)
X_test_ohe = preprocessor.transform(X_test)

In [84]:
# Select one of the below classifiers.
clf = XGBClassifier(min_child_weight=0.5, max_depth=3, gamma=0.2)
# clf = LogisticRegression(C=10)
# clf = DecisionTreeClassifier(max_depth=10, min_samples_split=5)
# clf = RandomForestClassifier(max_depth=15, min_samples_split=10, n_estimators=50)

# Fit the classifier.
clf.fit(X_train_ohe, Y_train)
# Define prediction function.
predictor = lambda x: clf.predict_proba(preprocessor.transform(x))

# Compute accuracy.
acc = accuracy_score(y_true=Y_test, y_pred=predictor(X_test).argmax(axis=1))
print("Accuracy: %.3f" % acc)

Accuracy: 0.720


In [85]:
# Define attribute types, required for datatype conversion.
feature_types = {"Age": int}

# Define data preprocessor and inverse preprocessor. The invers preprocessor include datatype conversions.
heae_preprocessor, heae_inv_preprocessor = get_he_preprocessor(X=X_train,
                                                               feature_names=german.feature_names,
                                                               category_map=german.category_map,
                                                               feature_types=feature_types)

# Define trainset
trainset_input = heae_preprocessor(X_train).astype(np.float32)
trainset_outputs = {
    "output_1": trainset_input[:, :len(numerical_ids)]
}

for i, cat_id in enumerate(categorical_ids):
    trainset_outputs.update({
        f"output_{i+2}": X_train[:, cat_id]
    })
    
trainset = tf.data.Dataset.from_tensor_slices((trainset_input, trainset_outputs))
trainset = trainset.shuffle(1024).batch(128, drop_remainder=True)

In [86]:
# Define autoencoder path and create dir if it doesn't exist.
heae_path = os.path.join("tensorflow", "German_autoencoder")
if not os.path.exists(heae_path):
    os.makedirs(heae_path)

# Define constants.
EPOCHS = 50              # epochs to train the autoencoder
HIDDEN_DIM = 128         # hidden dimension of the autoencoder
LATENT_DIM = 15          # define latent dimension

# Define output dimensions.
OUTPUT_DIMS = [len(numerical_ids)]
OUTPUT_DIMS += [len(german.category_map[cat_id]) for cat_id in categorical_ids]

# Define the heterogeneous auto-encoder.
heae = HeAE(encoder=GermanCreditEncoder(hidden_dim=HIDDEN_DIM, latent_dim=LATENT_DIM),
            decoder=GermanCreditDecoder(hidden_dim=HIDDEN_DIM, output_dims=OUTPUT_DIMS))

# Define loss functions.
he_loss = [keras.losses.MeanSquaredError()]
he_loss_weights = [1.]

# Add categorical losses.
for i in range(len(categorical_names)):
    he_loss.append(keras.losses.SparseCategoricalCrossentropy(from_logits=True))
    he_loss_weights.append(1./len(categorical_names))

# Define metrics.
metrics = {}
for i, cat_name in enumerate(categorical_names):
    metrics.update({f"output_{i+2}": keras.metrics.SparseCategoricalAccuracy()})
    
# Compile model.
heae.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
             loss=he_loss,
             loss_weights=he_loss_weights,
             metrics=metrics)

if len(os.listdir(heae_path)) == 0:
    # Fit and save autoencoder.
    heae.fit(trainset, epochs=EPOCHS)
    heae.save(heae_path, save_format="tf")
else:
    # Load the model.
    heae = keras.models.load_model(heae_path, compile=False)

In [87]:
# Define constants
COEFF_SPARSITY = 0.5               # sparisty coefficient
COEFF_CONSISTENCY = 0.5            # consisteny coefficient
TRAIN_STEPS = 10000                # number of training steps -> consider increasing the number of steps
BATCH_SIZE = 100                   # batch size

In [89]:
# Define immutable features.
immutable_features = ["Age",'PersonalStatusSex',"Purpose","PeopleLiableToProvideMaintenance"]

# Define ranges. This means that the `Age` feature can not decrease.
ranges = {'Age': [0.0, 1.0]}

In [90]:
explainer = CounterfactualRLTabular(predictor=predictor,
                                    encoder=heae.encoder,
                                    decoder=heae.decoder,
                                    latent_dim=LATENT_DIM,
                                    encoder_preprocessor=heae_preprocessor,
                                    decoder_inv_preprocessor=heae_inv_preprocessor,
                                    coeff_sparsity=COEFF_SPARSITY,
                                    coeff_consistency=COEFF_CONSISTENCY,
                                    category_map=german.category_map,
                                    feature_names=german.feature_names,
                                    ranges=ranges,
                                    immutable_features=immutable_features,
                                    train_steps=TRAIN_STEPS,
                                    batch_size=BATCH_SIZE,
                                    backend="tensorflow")

In [91]:
# Fit the explainer.
explainer = explainer.fit(X=X_train)

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [16:19<00:00, 10.21it/s]


In [92]:
# Select some positive examples.
X_positive = X_test[np.argmax(predictor(X_test), axis=1) == 1]

X = X_positive[:1000]
Y_t = np.array([0])
C = []
# Generate counterfactual instances.
explanation = explainer.explain(X, Y_t, C)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.58it/s]


In [93]:
# Concat labels to the original instances.
orig = np.concatenate(
    [explanation.data['orig']['X'], explanation.data['orig']['class']],
    axis=1
)

# Concat labels to the counterfactual instances.
cf = np.concatenate(
    [explanation.data['cf']['X'], explanation.data['cf']['class']],
    axis=1
)

# Define new feature names and category map by including the label.
feature_names = german.feature_names + ["Label"]
category_map = deepcopy(german.category_map)
category_map.update({feature_names.index("Label"): german.target_names})

# Replace label encodings with strings.
orig_pd = pd.DataFrame(
    apply_category_mapping(orig, category_map),
    columns=feature_names
)

cf_pd = pd.DataFrame(
    apply_category_mapping(cf, category_map),
    columns=feature_names
)

In [94]:
orig_pd.head(n=10)

,ExistingChecking,Duration,CreditHistory,Purpose,CreditAmount,SavingsAccount,EmploymentSince,InstallmentRatePercentage,PersonalStatusSex,OtherDebtors,...,Property,Age,OtherInstallmentPlans,Housing,ExistingCreditsAtBank,Job,PeopleLiableToProvideMaintenance,Telephone,ForeignWorker,Label
0,A12,48,A30,A40,8358,A63,A72,1,A92,A101,...,A123,30,A143,A152,2,A173,1,A191,A201,Bad
1,A12,36,A32,A41,9398,A61,A72,1,A94,A101,...,A123,28,A143,A151,1,A174,1,A192,A201,Bad
2,A13,42,A30,A49,6289,A61,A72,2,A91,A101,...,A122,33,A143,A152,2,A173,1,A191,A201,Bad
3,A12,24,A33,A40,1965,A65,A73,4,A92,A101,...,A123,42,A143,A151,2,A173,1,A192,A201,Bad
4,A12,30,A32,A42,3441,A62,A73,2,A92,A102,...,A123,21,A143,A151,1,A173,1,A191,A201,Bad
5,A11,18,A33,A46,8471,A65,A73,1,A92,A101,...,A123,23,A143,A151,2,A173,1,A192,A201,Bad
6,A13,12,A33,A40,2247,A61,A73,2,A92,A101,...,A123,36,A142,A152,2,A173,1,A192,A201,Bad
7,A11,24,A33,A40,1333,A61,A71,4,A93,A101,...,A121,43,A143,A153,2,A173,2,A191,A201,Bad
8,A12,24,A32,A41,11560,A61,A73,1,A92,A101,...,A123,23,A143,A151,2,A174,1,A191,A201,Bad
9,A11,24,A31,A40,2325,A62,A74,2,A93,A101,...,A123,32,A141,A152,1,A173,1,A191,A201,Bad


In [95]:
cf_pd.head(n=10)

,ExistingChecking,Duration,CreditHistory,Purpose,CreditAmount,SavingsAccount,EmploymentSince,InstallmentRatePercentage,PersonalStatusSex,OtherDebtors,...,Property,Age,OtherInstallmentPlans,Housing,ExistingCreditsAtBank,Job,PeopleLiableToProvideMaintenance,Telephone,ForeignWorker,Label
0,A14,41.927198,A34,A40,7628.104066,A61,A73,1.0,A92,A101,...,A123,30,A143,A152,1.89534,A173,1.0,A191,A201,Good
1,A14,38.498512,A32,A41,7508.232591,A65,A73,1.249267,A94,A101,...,A123,28,A143,A151,1.032459,A173,1.0,A192,A201,Good
2,A14,43.320482,A34,A49,5281.479491,A61,A73,1.309512,A91,A101,...,A121,33,A143,A152,1.988977,A173,1.0,A191,A201,Good
3,A14,25.009721,A34,A40,2718.503728,A65,A75,3.868753,A92,A101,...,A121,42,A143,A152,2.088741,A173,1.0,A192,A201,Good
4,A14,25.601481,A32,A42,3466.551474,A65,A73,1.563133,A92,A101,...,A121,21,A143,A151,1.010803,A173,1.0,A191,A201,Good
5,A14,21.855818,A34,A46,7900.549585,A65,A73,1.52429,A92,A101,...,A121,23,A143,A152,1.725049,A173,1.0,A192,A201,Good
6,A14,8.431014,A34,A40,2923.173878,A61,A74,2.061147,A92,A101,...,A123,36,A143,A152,2.024471,A173,1.0,A192,A201,Good
7,A14,12.764926,A34,A40,1700.541606,A65,A73,3.989876,A93,A101,...,A121,43,A143,A152,1.904548,A173,2.0,A191,A201,Good
8,A14,21.931635,A32,A41,7098.102551,A65,A73,1.0,A92,A101,...,A122,23,A143,A152,1.405385,A173,1.0,A191,A201,Good
9,A14,22.063396,A32,A40,2472.77385,A62,A73,1.963125,A93,A101,...,A121,32,A143,A152,1.0,A173,1.0,A191,A201,Good


In [96]:
# Generate counterfactual instances.
X = X_positive[0].reshape(1, -1)
explanation = explainer.explain(X=X, Y_t=Y_t, C=C, diversity=True, num_samples=100, batch_size=10)

13it [00:00, 16.11it/s]


In [97]:
# Concat label column.
orig = np.concatenate(
    [explanation.data['orig']['X'], explanation.data['orig']['class']],
    axis=1
)

cf = np.concatenate(
    [explanation.data['cf']['X'], explanation.data['cf']['class']],
    axis=1
)

# Transfrom label encodings to string.
orig_pd = pd.DataFrame(
    apply_category_mapping(orig, category_map),
    columns=feature_names,
)

cf_pd = pd.DataFrame(
    apply_category_mapping(cf, category_map),
    columns=feature_names,
)

In [98]:
orig_pd.head(n=5)

,ExistingChecking,Duration,CreditHistory,Purpose,CreditAmount,SavingsAccount,EmploymentSince,InstallmentRatePercentage,PersonalStatusSex,OtherDebtors,...,Property,Age,OtherInstallmentPlans,Housing,ExistingCreditsAtBank,Job,PeopleLiableToProvideMaintenance,Telephone,ForeignWorker,Label
0,A12,48,A30,A40,8358,A63,A72,1,A92,A101,...,A123,30,A143,A152,2,A173,1,A191,A201,Bad


In [99]:
cf_pd.head(n=5)

,ExistingChecking,Duration,CreditHistory,Purpose,CreditAmount,SavingsAccount,EmploymentSince,InstallmentRatePercentage,PersonalStatusSex,OtherDebtors,...,Property,Age,OtherInstallmentPlans,Housing,ExistingCreditsAtBank,Job,PeopleLiableToProvideMaintenance,Telephone,ForeignWorker,Label
0,A12,9.017365,A33,A40,7467.462048,A61,A73,1.0,A92,A101,...,A123,30,A143,A152,1.786473,A173,1.0,A191,A201,Good
1,A12,9.738077,A32,A40,7660.262127,A61,A73,1.0,A92,A101,...,A123,30,A143,A152,1.828675,A173,1.0,A191,A201,Good
2,A12,10.561105,A33,A40,7641.350945,A61,A73,1.0,A92,A101,...,A121,30,A143,A152,1.773576,A173,1.0,A191,A201,Good
3,A12,12.770822,A33,A40,7473.775909,A61,A74,1.0,A92,A101,...,A123,30,A143,A152,1.821425,A173,1.0,A191,A201,Good
4,A12,12.829354,A32,A40,7700.594901,A61,A72,1.0,A92,A101,...,A121,30,A143,A152,1.794864,A173,1.0,A191,A201,Good


In [187]:
counterfactuals_list_cfrl = []

# Assuming X_positive, Y_t, C, category_map, feature_names are already defined
for i in range(X_positive.shape[0]):
    X = X_positive[i].reshape(1, -1)
    explanation = explainer.explain(X=X, Y_t=Y_t, C=C, diversity=True, num_samples=100, batch_size=10)
    
    # Concat label column.
    orig = np.concatenate(
        [explanation.data['orig']['X'], explanation.data['orig']['class']],
        axis=1
    )

    cf = np.concatenate(
        [explanation.data['cf']['X'], explanation.data['cf']['class']],
        axis=1
    )

    # Transform label encodings to string.
    orig_pd = pd.DataFrame(
        apply_category_mapping(orig, category_map),
        columns=feature_names,
    )

    cf_pd = pd.DataFrame(
        apply_category_mapping(cf, category_map),
        columns=feature_names,
    )
    
    counterfactuals_list_cfrl.append((orig_pd, cf_pd))


12it [00:00, 15.97it/s]
11it [00:00, 18.68it/s]
13it [00:00, 19.16it/s]
10it [00:00, 18.42it/s]
13it [00:00, 18.21it/s]
20it [00:01, 17.87it/s]
10it [00:00, 18.33it/s]
11it [00:00, 18.55it/s]
11it [00:00, 16.92it/s]
12it [00:00, 18.31it/s]
10it [00:00, 16.78it/s]
11it [00:00, 18.30it/s]
16it [00:00, 18.11it/s]
14it [00:00, 17.95it/s]
12it [00:00, 18.47it/s]
12it [00:00, 18.50it/s]
13it [00:00, 18.18it/s]
13it [00:00, 18.42it/s]
10it [00:00, 18.54it/s]
11it [00:00, 18.05it/s]
13it [00:00, 18.69it/s]
11it [00:00, 18.66it/s]
12it [00:00, 18.66it/s]
11it [00:00, 19.15it/s]
13it [00:00, 19.15it/s]
10it [00:00, 19.28it/s]
12it [00:00, 18.08it/s]
11it [00:00, 19.04it/s]
11it [00:00, 19.19it/s]
11it [00:00, 18.96it/s]
11it [00:00, 18.65it/s]
11it [00:00, 18.61it/s]
11it [00:00, 19.30it/s]
10it [00:00, 19.37it/s]
15it [00:00, 19.07it/s]
16it [00:00, 19.26it/s]
11it [00:00, 19.09it/s]
11it [00:00, 18.96it/s]
11it [00:00, 18.39it/s]
18it [00:00, 18.88it/s]
12it [00:00, 18.52it/s]
11it [00:00, 18.

In [191]:
counterfactuals_list_cfrl

[(  ExistingChecking Duration CreditHistory Purpose CreditAmount SavingsAccount  \
  0              A12       48           A30     A40         8358            A63   
  
    EmploymentSince InstallmentRatePercentage PersonalStatusSex OtherDebtors  \
  0             A72                         1               A92         A101   
  
     ... Property Age OtherInstallmentPlans Housing ExistingCreditsAtBank   Job  \
  0  ...     A123  30                  A143    A152                     2  A173   
  
    PeopleLiableToProvideMaintenance Telephone ForeignWorker Label  
  0                                1      A191          A201   Bad  
  
  [1 rows x 21 columns],
     ExistingChecking   Duration CreditHistory Purpose CreditAmount  \
  0               A12  10.941309           A30     A40  7517.195354   
  1               A12  12.236654           A33     A40  7468.741574   
  2               A12   12.53638           A34     A40  7572.068721   
  3               A12  13.206203           A34   

In [204]:
trained_num_transf = preprocessor.named_transformers_["num"]
metrics = calculate_metrics(counterfactuals_list_cfrl, df, numerical_names, 
                            immutable_features, non_decreasing_features, 
                            correlated_features, preprocessor, method='random')
print(metrics)

X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names


  ExistingChecking Duration CreditHistory Purpose CreditAmount SavingsAccount  \
0              A12       48           A30     A40         8358            A63   

  EmploymentSince InstallmentRatePercentage PersonalStatusSex OtherDebtors  \
0             A72                         1               A92         A101   

  PresentResidenceSince Property Age OtherInstallmentPlans Housing  \
0                     1     A123  30                  A143    A152   

  ExistingCreditsAtBank   Job PeopleLiableToProvideMaintenance Telephone  \
0                     2  A173                                1      A191   

  ForeignWorker  
0          A201  
  ExistingChecking Duration CreditHistory Purpose CreditAmount SavingsAccount  \
0              A12       36           A32     A41         9398            A61   

  EmploymentSince InstallmentRatePercentage PersonalStatusSex OtherDebtors  \
0             A72                         1               A94         A101   

  PresentResidenceSince Proper

X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names


  ExistingChecking Duration CreditHistory Purpose CreditAmount SavingsAccount  \
0              A13       42           A30     A49         6289            A61   

  EmploymentSince InstallmentRatePercentage PersonalStatusSex OtherDebtors  \
0             A72                         2               A91         A101   

  PresentResidenceSince Property Age OtherInstallmentPlans Housing  \
0                     1     A122  33                  A143    A152   

  ExistingCreditsAtBank   Job PeopleLiableToProvideMaintenance Telephone  \
0                     2  A173                                1      A191   

  ForeignWorker  
0          A201  
  ExistingChecking Duration CreditHistory Purpose CreditAmount SavingsAccount  \
0              A12       24           A33     A40         1965            A65   

  EmploymentSince InstallmentRatePercentage PersonalStatusSex OtherDebtors  \
0             A73                         4               A92         A101   

  PresentResidenceSince Proper

X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names


...

X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but St

  ExistingChecking Duration CreditHistory Purpose CreditAmount SavingsAccount  \
0              A13       15           A32     A46          392            A61   

  EmploymentSince InstallmentRatePercentage PersonalStatusSex OtherDebtors  \
0             A72                         4               A92         A101   

  PresentResidenceSince Property Age OtherInstallmentPlans Housing  \
0                     4     A122  23                  A143    A151   

  ExistingCreditsAtBank   Job PeopleLiableToProvideMaintenance Telephone  \
0                     1  A173                                1      A192   

  ForeignWorker  
0          A201  
  ExistingChecking Duration CreditHistory Purpose CreditAmount SavingsAccount  \
0              A11       30           A30     A42         4583            A61   

  EmploymentSince InstallmentRatePercentage PersonalStatusSex OtherDebtors  \
0             A73                         2               A91         A103   

  PresentResidenceSince Proper

X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but St

  ExistingChecking Duration CreditHistory Purpose CreditAmount SavingsAccount  \
0              A12       48           A34     A42         5096            A61   

  EmploymentSince InstallmentRatePercentage PersonalStatusSex OtherDebtors  \
0             A73                         2               A92         A101   

  PresentResidenceSince Property Age OtherInstallmentPlans Housing  \
0                     3     A123  30                  A143    A152   

  ExistingCreditsAtBank   Job PeopleLiableToProvideMaintenance Telephone  \
0                     1  A174                                1      A192   

  ForeignWorker  
0          A201  
  ExistingChecking Duration CreditHistory Purpose CreditAmount SavingsAccount  \
0              A13       15           A32     A46         1905            A61   

  EmploymentSince InstallmentRatePercentage PersonalStatusSex OtherDebtors  \
0             A75                         4               A93         A101   

  PresentResidenceSince Proper

X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names


# DiCE

In [240]:
import dice_ml

# Combine data and target into a single dataframe
df = pd.DataFrame(german.data, columns=german.feature_names)
df['Outcome'] = german.target

# Define DiCE data object
d = dice_ml.Data(dataframe=df, 
                 continuous_features=numerical_names, 
                 outcome_name="Outcome")

# Define DiCE model object
backend = "sklearn"

class ModelWrapper:
    def __init__(self, predictor_func):
        self.predictor_func = predictor_func
    
    def predict_proba(self, instances):
        return self.predictor_func(instances)
    
    def predict(self, instances):
        # 获得概率预测
        prob_preds = self.predictor_func(instances)
        # 返回最大概率的类标签
        return np.argmax(prob_preds, axis=1)

# 包装我们的predictor函数
model_wrapper = ModelWrapper(predictor)

# 使用包装后的模型来定义DiCE的模型对象
m = dice_ml.Model(model=model_wrapper, backend=backend)
method = 'kdtree'

# 用之前的代码生成反事实
exp = dice_ml.Dice(d, m, method=method)
query_instance_df = pd.DataFrame([X_positive[0]], columns=german.feature_names)
dice_exp = exp.generate_counterfactuals(query_instance_df, total_CFs=5, desired_class="opposite")

# 可视化生成的反事实
dice_exp.visualize_as_dataframe()


X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has featu

Query instance (original outcome : 1)


,ExistingChecking,Duration,CreditHistory,Purpose,CreditAmount,SavingsAccount,EmploymentSince,InstallmentRatePercentage,PersonalStatusSex,OtherDebtors,...,Property,Age,OtherInstallmentPlans,Housing,ExistingCreditsAtBank,Job,PeopleLiableToProvideMaintenance,Telephone,ForeignWorker,Outcome
0,1,48,0,0,8358,2,1,1,1,0,...,2,30,2,1,2,2,1,0,0,1



Diverse Counterfactual set (new outcome: 0)


,ExistingChecking,Duration,CreditHistory,Purpose,CreditAmount,SavingsAccount,EmploymentSince,InstallmentRatePercentage,PersonalStatusSex,OtherDebtors,...,Property,Age,OtherInstallmentPlans,Housing,ExistingCreditsAtBank,Job,PeopleLiableToProvideMaintenance,Telephone,ForeignWorker,Outcome
980,1,30.0,4,3,8358.0,0,3,2.0,2,0,...,1,49.0,2,1,1.0,2,1.0,0,0,0
870,3,36.0,3,3,7678.0,2,3,2.0,1,0,...,2,40.0,2,1,2.0,2,1.0,1,0,0
108,0,24.0,2,3,7721.0,4,1,1.0,1,0,...,1,30.0,2,1,1.0,2,1.0,1,1,0
153,1,24.0,4,1,7758.0,3,4,2.0,1,0,...,3,30.0,2,0,1.0,2,1.0,0,0,0
804,1,12.0,2,0,7472.0,4,0,1.0,1,0,...,0,24.0,2,0,1.0,0,1.0,0,0,0


In [241]:
counterfactuals_list = []

query_instance_df = pd.DataFrame(X_positive, columns=german.feature_names)

for _, instance in query_instance_df.iterrows():
    instance_df = pd.DataFrame(instance).T
    dice_exp = exp.generate_counterfactuals(instance_df, total_CFs=5, desired_class="opposite")
    cf_df = dice_exp.visualize_as_dataframe(show_only_changes=False)
    if hasattr(dice_exp, 'cf_examples_list') and dice_exp.cf_examples_list:
        cf_df = dice_exp.cf_examples_list[0].final_cfs_df
        if cf_df is not None:
            counterfactuals_list.append((instance, cf_df))
            print("1")
        else:
            counterfactuals_list.append((instance, None))
            print("0")
    else:
        counterfactuals_list.append((instance, None))
        print("0")


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has featu

Query instance (original outcome : 1)


,ExistingChecking,Duration,CreditHistory,Purpose,CreditAmount,SavingsAccount,EmploymentSince,InstallmentRatePercentage,PersonalStatusSex,OtherDebtors,...,Property,Age,OtherInstallmentPlans,Housing,ExistingCreditsAtBank,Job,PeopleLiableToProvideMaintenance,Telephone,ForeignWorker,Outcome
0,1,48,0,0,8358,2,1,1,1,0,...,2,30,2,1,2,2,1,0,0,1



Diverse Counterfactual set (new outcome: 0)


,ExistingChecking,Duration,CreditHistory,Purpose,CreditAmount,SavingsAccount,EmploymentSince,InstallmentRatePercentage,PersonalStatusSex,OtherDebtors,...,Property,Age,OtherInstallmentPlans,Housing,ExistingCreditsAtBank,Job,PeopleLiableToProvideMaintenance,Telephone,ForeignWorker,Outcome
980,1,30.0,4,3,8358.0,0,3,2.0,2,0,...,1,49.0,2,1,1.0,2,1.0,0,0,0
870,3,36.0,3,3,7678.0,2,3,2.0,1,0,...,2,40.0,2,1,2.0,2,1.0,1,0,0
108,0,24.0,2,3,7721.0,4,1,1.0,1,0,...,1,30.0,2,1,1.0,2,1.0,1,1,0
153,1,24.0,4,1,7758.0,3,4,2.0,1,0,...,3,30.0,2,0,1.0,2,1.0,0,0,0
804,1,12.0,2,0,7472.0,4,0,1.0,1,0,...,0,24.0,2,0,1.0,0,1.0,0,0,0


...

Query instance (original outcome : 0)


,ExistingChecking,Duration,CreditHistory,Purpose,CreditAmount,SavingsAccount,EmploymentSince,InstallmentRatePercentage,PersonalStatusSex,OtherDebtors,...,Property,Age,OtherInstallmentPlans,Housing,ExistingCreditsAtBank,Job,PeopleLiableToProvideMaintenance,Telephone,ForeignWorker,Outcome
0,2,15,2,7,1905,0,4,4,2,0,...,2,40,2,0,1,3,1,1,0,0



Diverse Counterfactual set (new outcome: 1)


,ExistingChecking,Duration,CreditHistory,Purpose,CreditAmount,SavingsAccount,EmploymentSince,InstallmentRatePercentage,PersonalStatusSex,OtherDebtors,...,Property,Age,OtherInstallmentPlans,Housing,ExistingCreditsAtBank,Job,PeopleLiableToProvideMaintenance,Telephone,ForeignWorker,Outcome
74,0,36.0,2,7,1905.0,4,4,4.0,2,0,...,3,40.0,2,1,1.0,3,1.0,1,0,1
213,2,30.0,3,9,1905.0,0,4,4.0,2,0,...,0,66.0,2,1,1.0,3,1.0,1,0,1
62,1,36.0,0,9,1905.0,0,4,4.0,2,0,...,3,61.0,2,2,1.0,3,1.0,1,0,1
446,0,36.0,2,0,1905.0,0,1,4.0,1,0,...,2,34.0,2,1,1.0,2,1.0,1,0,1
998,0,45.0,2,4,1905.0,0,2,4.0,2,0,...,3,23.0,2,2,1.0,2,1.0,1,0,1


1


In [245]:
def find_proximity_cont(cfe, original_datapoint, continuous_features, mads):
    diff = original_datapoint[continuous_features].to_numpy() - cfe[continuous_features].to_numpy()
    dist_cont = np.mean(np.divide(np.abs(diff), mads))
    sparsity_cont = diff[0].nonzero()[0].shape[0]
    return dist_cont, sparsity_cont


# def find_proximity_cat(cfe, original_datapoint, categorical_features):
#     print("Columns in cfe inside find_proximity_cat:", cfe.columns)
#     cfe_cats = cfe[categorical_features].to_numpy().astype(float)
#     diff = original_datapoint[categorical_features].to_numpy() - cfe_cats
#     sparsity_cat = diff[0].nonzero()[0].shape[0]
#     dist_cat = sparsity_cat * 1.0 / len(categorical_features)
#     return dist_cat, sparsity_cat

def find_proximity_cat(cfe, original_datapoint, encoded_columns):
    """
    cfe: Counterfactual example after one-hot encoding.
    original_datapoint: Original data point after one-hot encoding.
    encoded_columns: List of columns after one-hot encoding.
    """
    sparsity_cat = 0
    total_features = len(encoded_columns)

    # 计算差异
    diff = original_datapoint[encoded_columns].to_numpy() - cfe[encoded_columns].to_numpy()
    
    # 检查是否有差异
    sparsity_cat = np.sum(np.any(diff != 0, axis=1))

    dist_cat = sparsity_cat * 1.0 / total_features
    return dist_cat, sparsity_cat




# def follows_causality(cfe, original_datapoint, immutable_features, 
#         non_decreasing_features, correlated_features, method):
#     follows = True
#     diff = cfe.to_numpy().astype(float) - original_datapoint.to_numpy()
#     m2 = (diff != 0)[0].nonzero()
#     changed_columns = cfe.columns[m2].tolist()
    
#     # Check immutable features
#     changed_immutable_features = set(changed_columns).intersection(set(immutable_features))
#     if changed_immutable_features:
#         print("Warning: The following immutable features were changed:", changed_immutable_features)
#         if not any(x in method for x in ["dice-vae", "MACE", "CFproto"]):
#             follows = False
#             return follows

#     # Check non-decreasing features
#     diff_nondecrease = cfe[non_decreasing_features].to_numpy().astype(float) - original_datapoint[non_decreasing_features].to_numpy()
#     m2 = (diff_nondecrease < 0)[0].nonzero()
#     if m2[0].shape[0] > 0:
#         follows = False
#         return follows
    
#     # Check correlated features
#     for f1, f2, linear_add in correlated_features:
#         seq_f1 = cfe.columns.tolist().index(f1)
#         seq_f2 = cfe.columns.tolist().index(f2)
#         if diff[0][seq_f1] > 0:  # If there is an increase in f1
#             if not diff[0][seq_f2] >= linear_add:  # then there must be an increase in f2 of value 'linear_add'
#                 follows = False
#                 return follows

#     return follows

def get_encoded_columns(original_column, cfe_columns):
    """Get the OneHotEncoded column names for a given original column."""
    return [col for col in cfe_columns if col.startswith(original_column + "_")]

def follows_causality(cfe, original_datapoint, immutable_features, 
        non_decreasing_features, correlated_features, method):
    follows = True
    diff = cfe.to_numpy().astype(float) - original_datapoint.to_numpy()
    m2 = (diff != 0)[0].nonzero()
    changed_columns = cfe.columns[m2].tolist()
    
    # Convert immutable features to their encoded names
    immutable_features_encoded = []
    for feature in immutable_features:
        immutable_features_encoded.extend(get_encoded_columns(feature, cfe.columns))
    
    # Check immutable features
    changed_immutable_features = set(changed_columns).intersection(set(immutable_features_encoded))
    if changed_immutable_features:
        print("Warning: The following immutable features were changed:", changed_immutable_features)
        if not any(x in method for x in ["dice-vae", "MACE", "CFproto"]):
            follows = False
            return follows

    # Convert non-decreasing features to their encoded names
    non_decreasing_features_encoded = []
    for feature in non_decreasing_features:
        non_decreasing_features_encoded.extend(get_encoded_columns(feature, cfe.columns))
    
    # Check non-decreasing features
    diff_nondecrease = cfe[non_decreasing_features_encoded].to_numpy().astype(float) - original_datapoint[non_decreasing_features_encoded].to_numpy()
    m2 = (diff_nondecrease < 0)[0].nonzero()
    if m2[0].shape[0] > 0:
        follows = False
        return follows
    
    # Check correlated features
    for f1, f2, linear_add in correlated_features:
        f1_encoded = get_encoded_columns(f1, cfe.columns)
        f2_encoded = get_encoded_columns(f2, cfe.columns)
        for col1 in f1_encoded:
            seq_f1 = cfe.columns.tolist().index(col1)
            for col2 in f2_encoded:
                seq_f2 = cfe.columns.tolist().index(col2)
                if diff[0][seq_f1] > 0:  # If there is an increase in f1
                    if not diff[0][seq_f2] >= linear_add:  # then there must be an increase in f2 of value 'linear_add'
                        follows = False
                        return follows

    return follows


def find_manifold_dist(cfe, knn):
    nearest_dist, nearest_points = knn.kneighbors(cfe.to_numpy(), 1, return_distance=True)
    quantity = np.mean(nearest_dist)		# Now we have that lambda, so no need to divide by self.no_points 
    return quantity

def calculate_metrics(counterfactuals_list, dataset, continuous_features, 
                      immutable_features, non_decreasing_features, 
                      correlated_features, scaler, method):
    
     # 获取OneHotEncoder的输出列名
    encoder = scaler.named_transformers_['cat']
    categorical_features = dataset.columns[categorical_ids].tolist()
    encoded_columns = encoder.get_feature_names_out(input_features=categorical_features).tolist()
    
    # 组合连续特征和编码后的分类特征的列名
    all_transformed_columns = continuous_features + encoded_columns
    
    # 1. Transform the dataset
    transformed_dataset = scaler.transform(dataset.drop(columns=['Outcome']))
    transformed_dataset = pd.DataFrame(transformed_dataset, columns=all_transformed_columns)
    
    # 2. Calculate normalized MADS for continuous features
    normalized_mads = {}
    for feature in continuous_features:
        normalized_mads[feature] = np.median(abs(transformed_dataset[feature].values - np.median(transformed_dataset[feature].values)))
    mads = [normalized_mads[key] if normalized_mads[key] != 0.0 else 1.0 for key in normalized_mads]
    
    # 3. Setup KNN for manifold distance
    knn = NearestNeighbors(n_neighbors=5, p=1)
    knn.fit(transformed_dataset)
    
    # 4. Loop through counterfactuals to calculate metrics
    avg_proximity_cont = []
    avg_proximity_cat = []
    avg_sparsity = []
    avg_causality = []
    avg_manifold_dist = []
    
#     for instance_series, cf_df in counterfactuals_list:
#         instance_df = pd.DataFrame(instance_series).T
#     # Transform data
#         cf_df = cf_df.iloc[:, :-1] 
#         cfe = cf_df 
#         cfe_transformed = scaler.transform(cf_df)
#         instance_df_transformed = scaler.transform(instance_df)
#         cfe_transformed = pd.DataFrame(cfe_transformed, columns=all_transformed_columns)
#         instance_df_transformed = pd.DataFrame(instance_df_transformed, columns=all_transformed_columns)

    for instance_series, cf_df in counterfactuals_list:  # kdtree 专属
        instance_df = pd.DataFrame(instance_series).T  # 修改的部分
        cfe = cf_df 
        cfe_transformed = scaler.transform(cf_df)
        instance_df_transformed = scaler.transform(instance_df)
        cfe_transformed = pd.DataFrame(cfe_transformed, columns=all_transformed_columns)
        instance_df_transformed = pd.DataFrame(instance_df_transformed, columns = all_transformed_columns )
        

        
        # Proximity
        proximity_cont, sparsity_cont = find_proximity_cont(cfe_transformed, instance_df_transformed, continuous_features, mads)
        proximity_cat, sparsity_cat = find_proximity_cat(cfe_transformed, instance_df_transformed, encoded_columns)
        
        # Sparsity
        sparsity = sparsity_cont + sparsity_cat
        
        # Causality
        causality = follows_causality(cfe_transformed, instance_df_transformed, immutable_features, 
                                      non_decreasing_features, correlated_features, method)
        
        # Manifold Distance
        manifold_dist = find_manifold_dist(cfe_transformed, knn)
        
        avg_proximity_cont.append(proximity_cont)
        avg_proximity_cat.append(proximity_cat)
        avg_sparsity.append(sparsity)
        avg_causality.append(causality)
        avg_manifold_dist.append(manifold_dist)
    
    # 5. Calculate average metrics
    avg_metrics = {
        "avg_proximity_cont": np.mean(avg_proximity_cont),
        "avg_proximity_cat": np.mean(avg_proximity_cat),
        "avg_sparsity": np.mean(avg_sparsity),
        "avg_causality": np.mean(avg_causality),
        "avg_manifold_dist": np.mean(avg_manifold_dist)
    }
    
    return avg_metrics


In [246]:
continuous_features = ['Duration', 'CreditAmount', 'InstallmentRatePercentage', 'PresentResidenceSince', 'Age', 'ExistingCreditsAtBank', 'PeopleLiableToProvideMaintenance']
immutable_features = ['PersonalStatusSex', 'PeopleLiableToProvideMaintenance', 'ForeignWorker', 'Purpose']
non_decreasing_features = ['Age', 'Job']
correlated_features = []

In [247]:
trained_num_transf = preprocessor.named_transformers_["num"]
metrics = calculate_metrics(counterfactuals_list, df, numerical_names, 
                            immutable_features, non_decreasing_features, 
                            correlated_features, preprocessor, method='random')
print(metrics)

X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names


X does not have valid feature names, but NearestNeighbors was fitted with feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names


X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names


X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names


X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names


...

X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names


X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X has feature names, but OneHotEncoder was fitted without feature names


X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names


X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X has feature names, but OneHotEncoder was fitted without feature names
X has feature names, but StandardScaler was fitted without feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names


{'avg_proximity_cont': 0.25776397515527955, 'avg_proximity_cat': 0.09259259259259257, 'avg_sparsity': 6.804347826086956, 'avg_causality': 0.0, 'avg_manifold_dist': 13.0}


In [ ]:
counterfactuals_list